In [ ]:
import marimo as mo

In [ ]:
import xdsl.utils.marimo as xmo

# Compiling `linalg` to Snitch

This notebook walks through compiling micro-kernels defined in `linalg` to RISC-V and RISC-V with extensions for [Snitch](https://pulp-platform.github.io/snitch/), a neural network accelerator.

_Toggle app view with `⌘` + `.` or `ctrl` + `.`_

In [ ]:
# Import all the necessary functionality from xDSL for this notebook
# If you see an error about xdsl not being defined run this cell manually

from xdsl.backend.riscv.lowering import (
    convert_arith_to_riscv,
    convert_func_to_riscv_func,
    convert_memref_to_riscv,
    convert_scf_to_riscv_scf,
)
from xdsl.backend.riscv.lowering.convert_riscv_scf_to_riscv_cf import (
    ConvertRiscvScfToRiscvCfPass,
)
from xdsl.backend.riscv.lowering.convert_snitch_stream_to_snitch import (
    ConvertSnitchStreamToSnitch,
)
from xdsl.builder import ImplicitBuilder
from xdsl.context import MLContext
from xdsl.dialects import arith, func, linalg
from xdsl.dialects.builtin import AffineMap, AffineMapAttr, MemRefType, ModuleOp, f64
from xdsl.dialects.riscv import riscv_code
from xdsl.interpreters.utils.ptr import TypedPtr
from xdsl.ir import Attribute, Block, Region, SSAValue
from xdsl.passes import PipelinePass
from xdsl.tools.command_line_tool import get_all_dialects
from xdsl.transforms import (
    arith_add_fastmath,
    convert_linalg_to_loops,
    convert_linalg_to_memref_stream,
    convert_memref_stream_to_loops,
    convert_memref_stream_to_snitch_stream,
    convert_riscv_scf_for_to_frep,
    dead_code_elimination,
    loop_hoist_memref,
    lower_affine,
    memref_streamify,
    reconcile_unrealized_casts,
)
from xdsl.transforms.canonicalize import CanonicalizePass
from xdsl.transforms.lower_snitch import LowerSnitchPass
from xdsl.transforms.mlir_opt import MLIROptPass
from xdsl.transforms.riscv_register_allocation import RISCVRegisterAllocation
from xdsl.transforms.riscv_scf_loop_range_folding import (
    RiscvScfLoopRangeFoldingPass,
)
from xdsl.transforms.snitch_register_allocation import SnitchRegisterAllocation

In [ ]:
a_type = MemRefType(f64, a_shape)
b_type = MemRefType(f64, b_shape)
c_type = MemRefType(f64, c_shape)

kernel_op = func.FuncOp("matmul", ((a_type, b_type, c_type), ()))
with ImplicitBuilder(kernel_op.body) as (a, b, c):
    # Add name hints to make it easier to track how values are lowered
    a.name_hint = "A"
    b.name_hint = "B"
    c.name_hint = "C"
    body = Region(Block(arg_types = (f64, f64, f64)))
    linalg.GenericOp(
        inputs=(a, b),
        outputs=(c,),
        body=body,
        indexing_maps=(
            AffineMapAttr(AffineMap.from_callable(lambda m, n, k: (m, k))),
            AffineMapAttr(AffineMap.from_callable(lambda m, n, k: (k, n))),
            AffineMapAttr(AffineMap.from_callable(lambda m, n, k: (m, n))),
        ),
        iterator_types=(
            linalg.IteratorTypeAttr.parallel(),
            linalg.IteratorTypeAttr.parallel(),
            linalg.IteratorTypeAttr.reduction(),
        )
    )
    with ImplicitBuilder(body) as (a_val, b_val, acc_old_val):
        prod_val = arith.MulfOp(a_val, b_val).result
        acc_new_val = arith.AddfOp(acc_old_val, prod_val).result
        linalg.YieldOp(acc_new_val)
        # Add more name hints to make it easier to track how values are lowered
        a_val.name_hint = "a"
        b_val.name_hint = "b"
        acc_old_val.name_hint = "acc_old"
        prod_val.name_hint = "prod"
        acc_new_val.name_hint = "acc_new"
    func.ReturnOp()

linalg_module = ModuleOp((kernel_op,))

mo.md(f"""

Here is matrix multiplication defined in the `linalg` dialect, with the iteration space decoupled from the computation:

{xmo.module_html(linalg_module)}
""")

Here is matrix multiplication defined in the linalg dialect, with the iteration space decoupled from the computation: 
 <marimo-code-editor data-initial-value='"builtin.module {\n func.func @matmul(%A : memref<2x2xf64>, %B : memref<2x2xf64>, %C : memref<2x2xf64>) {\n linalg.generic {indexing_maps = [affine_map<(d0, d1, d2) -> (d0, d2)>, affine_map<(d0, d1, d2) -> (d2, d1)>, affine_map<(d0, d1, d2) -> (d0, d1)>], iterator_types = [\"parallel\", \"parallel\", \"reduction\"]} ins(%A, %B : memref<2x2xf64>, memref<2x2xf64>) outs(%C : memref<2x2xf64>) {\n ^0(%a : f64, %b : f64, %acc_old : f64):\n %prod = arith.mulf %a, %b : f64\n %acc_new = arith.addf %acc_old, %prod : f64\n linalg.yield %acc_new : f64\n }\n func.return\n }\n}"' data-label='null' data-language='"javascript"' data-placeholder='""' data-disabled='true'>

In [ ]:
min_val = 1
max_val = 10
m = mo.ui.slider(min_val, max_val, value=2, label="M")
n = mo.ui.slider(min_val, max_val, value=2, label="N")
k = mo.ui.slider(min_val, max_val, value=2, label="K")

In [ ]:
mo.md(
    f"""
    We can parametrize the shapes of the matrices operated on:

    {m}{m.value}

    {n}{n.value}

    {k}{k.value}
    """
)

We can parametrize the shapes of the matrices operated on: 
 2 
 2 
 2

In [ ]:
a_shape = (m.value, k.value)
b_shape = (k.value, n.value)
c_shape = (m.value, n.value)

mo.md(
    f"""

    ```
    A: {'x'.join(str(dim) for dim in a_shape)}xf64 B: {'x'.join(str(dim) for dim in b_shape)}xf64 C: {'x'.join(str(dim) for dim in c_shape)}xf64
    ```
    """
)

A: 2x2xf64 B: 2x2xf64 C: 2x2xf64

### Compiling to RISC-V

In [ ]:
linalg_ctx = MLContext()

for dialect_name, dialect_factory in get_all_dialects().items():
    linalg_ctx.register_dialect(dialect_name, dialect_factory)

We can take this representation, and lower to RISC-V-specific dialects:

In [ ]:
lower_to_riscv = PipelinePass(
    [
        convert_linalg_to_loops.ConvertLinalgToLoopsPass(),
        convert_func_to_riscv_func.ConvertFuncToRiscvFuncPass(),
        convert_memref_to_riscv.ConvertMemrefToRiscvPass(),
        convert_arith_to_riscv.ConvertArithToRiscvPass(),
        convert_scf_to_riscv_scf.ConvertScfToRiscvPass(),
        reconcile_unrealized_casts.ReconcileUnrealizedCastsPass(),
    ]
)

riscv_ctx, riscv_module, riscv_html = xmo.pipeline_html(
    linalg_ctx, linalg_module, tuple(("", p) for p in lower_to_riscv.passes)
)

riscv_html

convert-linalg-to-loops <marimo-code-editor data-initial-value='"builtin.module {\n func.func @matmul(%A : memref<2x2xf64>, %B : memref<2x2xf64>, %C : memref<2x2xf64>) {\n %0 = arith.constant 2 : index\n %1 = arith.constant 2 : index\n %2 = arith.constant 2 : index\n %3 = arith.constant 0 : index\n %4 = arith.constant 1 : index\n scf.for %5 = %3 to %0 step %4 {\n scf.for %6 = %3 to %1 step %4 {\n scf.for %7 = %3 to %2 step %4 {\n %a = memref.load %A[%5, %7] : memref<2x2xf64>\n %b = memref.load %B[%7, %6] : memref<2x2xf64>\n %acc_old = memref.load %C[%5, %6] : memref<2x2xf64>\n %prod = arith.mulf %a, %b : f64\n %acc_new = arith.addf %acc_old, %prod : f64\n memref.store %acc_new, %C[%5, %6] : memref<2x2xf64>\n }\n }\n }\n func.return\n }\n}"' data-label='null' data-language='"javascript"' data-placeholder='""' data-disabled='true'> convert-func-to-riscv-func <marimo-code-editor data-initial-value='"builtin.module {\n riscv.assembly_section \".text\" {\n riscv.directive \".globl\" \"matmul\"\n riscv.directive \".p2align\" \"2\"\n riscv_func.func @matmul(%A : !riscv.reg<a0>, %B : !riscv.reg<a1>, %C : !riscv.reg<a2>) {\n %0 = riscv.mv %A : (!riscv.reg<a0>) -> !riscv.reg\n %A_1 = builtin.unrealized_conversion_cast %0 : !riscv.reg to memref<2x2xf64>\n %1 = riscv.mv %B : (!riscv.reg<a1>) -> !riscv.reg\n %B_1 = builtin.unrealized_conversion_cast %1 : !riscv.reg to memref<2x2xf64>\n %2 = riscv.mv %C : (!riscv.reg<a2>) -> !riscv.reg\n %C_1 = builtin.unrealized_conversion_cast %2 : !riscv.reg to memref<2x2xf64>\n %3 = arith.constant 2 : index\n %4 = arith.constant 2 : index\n %5 = arith.constant 2 : index\n %6 = arith.constant 0 : index\n %7 = arith.constant 1 : index\n scf.for %8 = %6 to %3 step %7 {\n scf.for %9 = %6 to %4 step %7 {\n scf.for %10 = %6 to %5 step %7 {\n %a = memref.load %A_1[%8, %10] : memref<2x2xf64>\n %b = memref.load %B_1[%10, %9] : memref<2x2xf64>\n %acc_old = memref.load %C_1[%8, %9] : memref<2x2xf64>\n %prod = arith.mulf %a, %b : f64\n %acc_new = arith.addf %acc_old, %prod : f64\n memref.store %acc_new, %C_1[%8, %9] : memref<2x2xf64>\n }\n }\n }\n riscv_func.return\n }\n }\n}"' data-label='null' data-language='"javascript"' data-placeholder='""' data-disabled='true'> convert-memref-to-riscv <marimo-code-editor data-initial-value='"builtin.module {\n riscv.assembly_section \".text\" {\n riscv.directive \".globl\" \"matmul\"\n riscv.directive \".p2align\" \"2\"\n riscv_func.func @matmul(%A : !riscv.reg<a0>, %B : !riscv.reg<a1>, %C : !riscv.reg<a2>) {\n %0 = riscv.mv %A : (!riscv.reg<a0>) -> !riscv.reg\n %A_1 = builtin.unrealized_conversion_cast %0 : !riscv.reg to memref<2x2xf64>\n %1 = riscv.mv %B : (!riscv.reg<a1>) -> !riscv.reg\n %B_1 = builtin.unrealized_conversion_cast %1 : !riscv.reg to memref<2x2xf64>\n %2 = riscv.mv %C : (!riscv.reg<a2>) -> !riscv.reg\n %C_1 = builtin.unrealized_conversion_cast %2 : !riscv.reg to memref<2x2xf64>\n %3 = arith.constant 2 : index\n %4 = arith.constant 2 : index\n %5 = arith.constant 2 : index\n %6 = arith.constant 0 : index\n %7 = arith.constant 1 : index\n scf.for %8 = %6 to %3 step %7 {\n scf.for %9 = %6 to %4 step %7 {\n scf.for %10 = %6 to %5 step %7 {\n %A_2 = builtin.unrealized_conversion_cast %A_1 : memref<2x2xf64> to !riscv.reg\n %11 = builtin.unrealized_conversion_cast %8 : index to !riscv.reg\n %12 = builtin.unrealized_conversion_cast %10 : index to !riscv.reg\n %pointer_dim_stride = riscv.li 2 : !riscv.reg\n %pointer_dim_offset = riscv.mul %11, %pointer_dim_stride : (!riscv.reg, !riscv.reg) -> !riscv.reg\n %pointer_offset = riscv.add %pointer_dim_offset, %12 : (!riscv.reg, !riscv.reg) -> !riscv.reg\n %bytes_per_element = riscv.li 8 : !riscv.reg\n %scaled_pointer_offset = riscv.mul %pointer_offset, %bytes_per_element {comment = \"multiply by element size\"} : (!riscv.reg, !riscv.reg) -> !riscv.reg\n %offset_pointer = riscv.add %A_2, %scaled_pointer_offset : (!riscv.reg, !riscv.reg) -> !riscv.reg\n %a = riscv.fld %offset_pointer, 0 {comment = \"load double from memref of

#### Register allocation

xDSL provides a register allocator for our RISC-V representation, that works on functions with structured control flow:

In [ ]:
allocate_registers = PipelinePass(
    [
        RISCVRegisterAllocation(),
        CanonicalizePass(),
    ]
)

regalloc_ctx, regalloc_module, regalloc_html = xmo.pipeline_html(
    riscv_ctx, riscv_module, tuple(("", p) for p in allocate_registers.passes),
)

regalloc_html

riscv-allocate-registers <marimo-code-editor data-initial-value='"builtin.module {\n riscv.assembly_section \".text\" {\n riscv.directive \".globl\" \"matmul\"\n riscv.directive \".p2align\" \"2\"\n riscv_func.func @matmul(%A : !riscv.reg<a0>, %B : !riscv.reg<a1>, %C : !riscv.reg<a2>) {\n %A_1 = riscv.mv %A : (!riscv.reg<a0>) -> !riscv.reg<t5>\n %B_1 = riscv.mv %B : (!riscv.reg<a1>) -> !riscv.reg<t4>\n %C_1 = riscv.mv %C : (!riscv.reg<a2>) -> !riscv.reg<t3>\n %0 = riscv.li 2 : !riscv.reg<a3>\n %1 = riscv.li 2 : !riscv.reg<t0>\n %2 = riscv.li 2 : !riscv.reg<t2>\n %3 = riscv.li 0 : !riscv.reg<zero>\n %4 = riscv.li 1 : !riscv.reg<t1>\n riscv_scf.for %5 : !riscv.reg<t6> = %3 to %0 step %4 {\n riscv_scf.for %6 : !riscv.reg<a4> = %3 to %1 step %4 {\n riscv_scf.for %7 : !riscv.reg<a5> = %3 to %2 step %4 {\n %pointer_dim_stride = riscv.li 2 : !riscv.reg<a6>\n %pointer_dim_offset = riscv.mul %5, %pointer_dim_stride : (!riscv.reg<t6>, !riscv.reg<a6>) -> !riscv.reg<a6>\n %pointer_offset = riscv.add %pointer_dim_offset, %7 : (!riscv.reg<a6>, !riscv.reg<a5>) -> !riscv.reg<a6>\n %bytes_per_element = riscv.li 8 : !riscv.reg<a7>\n %scaled_pointer_offset = riscv.mul %pointer_offset, %bytes_per_element {comment = \"multiply by element size\"} : (!riscv.reg<a6>, !riscv.reg<a7>) -> !riscv.reg<a6>\n %offset_pointer = riscv.add %A_1, %scaled_pointer_offset : (!riscv.reg<t5>, !riscv.reg<a6>) -> !riscv.reg<a6>\n %a = riscv.fld %offset_pointer, 0 {comment = \"load double from memref of shape (2, 2)\"} : (!riscv.reg<a6>) -> !riscv.freg<ft1>\n %pointer_dim_stride_1 = riscv.li 2 : !riscv.reg<a6>\n %pointer_dim_offset_1 = riscv.mul %7, %pointer_dim_stride_1 : (!riscv.reg<a5>, !riscv.reg<a6>) -> !riscv.reg<a6>\n %pointer_offset_1 = riscv.add %pointer_dim_offset_1, %6 : (!riscv.reg<a6>, !riscv.reg<a4>) -> !riscv.reg<a6>\n %bytes_per_element_1 = riscv.li 8 : !riscv.reg<a7>\n %scaled_pointer_offset_1 = riscv.mul %pointer_offset_1, %bytes_per_element_1 {comment = \"multiply by element size\"} : (!riscv.reg<a6>, !riscv.reg<a7>) -> !riscv.reg<a6>\n %offset_pointer_1 = riscv.add %B_1, %scaled_pointer_offset_1 : (!riscv.reg<t4>, !riscv.reg<a6>) -> !riscv.reg<a6>\n %b = riscv.fld %offset_pointer_1, 0 {comment = \"load double from memref of shape (2, 2)\"} : (!riscv.reg<a6>) -> !riscv.freg<ft2>\n %pointer_dim_stride_2 = riscv.li 2 : !riscv.reg<a6>\n %pointer_dim_offset_2 = riscv.mul %5, %pointer_dim_stride_2 : (!riscv.reg<t6>, !riscv.reg<a6>) -> !riscv.reg<a6>\n %pointer_offset_2 = riscv.add %pointer_dim_offset_2, %6 : (!riscv.reg<a6>, !riscv.reg<a4>) -> !riscv.reg<a6>\n %bytes_per_element_2 = riscv.li 8 : !riscv.reg<a7>\n %scaled_pointer_offset_2 = riscv.mul %pointer_offset_2, %bytes_per_element_2 {comment = \"multiply by element size\"} : (!riscv.reg<a6>, !riscv.reg<a7>) -> !riscv.reg<a6>\n %offset_pointer_2 = riscv.add %C_1, %scaled_pointer_offset_2 : (!riscv.reg<t3>, !riscv.reg<a6>) -> !riscv.reg<a6>\n %acc_old = riscv.fld %offset_pointer_2, 0 {comment = \"load double from memref of shape (2, 2)\"} : (!riscv.reg<a6>) -> !riscv.freg<ft0>\n %prod = riscv.fmul.d %a, %b : (!riscv.freg<ft1>, !riscv.freg<ft2>) -> !riscv.freg<ft1>\n %acc_new = riscv.fadd.d %acc_old, %prod : (!riscv.freg<ft0>, !riscv.freg<ft1>) -> !riscv.freg<ft0>\n %pointer_dim_stride_3 = riscv.li 2 : !riscv.reg<a6>\n %pointer_dim_offset_3 = riscv.mul %5, %pointer_dim_stride_3 : (!riscv.reg<t6>, !riscv.reg<a6>) -> !riscv.reg<a6>\n %pointer_offset_3 = riscv.add %pointer_dim_offset_3, %6 : (!riscv.reg<a6>, !riscv.reg<a4>) -> !riscv.reg<a6>\n %bytes_per_element_3 = riscv.li 8 : !riscv.reg<a7>\n %scaled_pointer_offset_3 = riscv.mul %pointer_offset_3, %bytes_per_element_3 {comment = \"multiply by element size\"} : (!riscv.reg<a6>, !riscv.reg<a7>) -> !riscv.reg<a6>\n %offset_pointer_3 = riscv.add %C_1, %scaled_pointer_offset_3 : (!riscv.reg<t3>, !riscv.reg<a6>) -> !riscv.reg<a6>\n riscv.fsd %offset_pointer_3, %acc_new, 0 {comment = \"store double value to memref of shape (2, 2)\"} : (!riscv.reg<a6>, !riscv.f

In [ ]:
lower_to_asm = PipelinePass(
    [
        ConvertRiscvScfToRiscvCfPass(),
        CanonicalizePass(),
    ]
)

riscv_asm_ctx, riscv_asm_module, assembly_html = xmo.pipeline_html(
    regalloc_ctx, regalloc_module, (("", lower_to_asm),),
)

assembly_html

convert-riscv-scf-to-riscv-cf,canonicalize <marimo-code-editor data-initial-value='"builtin.module {\n riscv.assembly_section \".text\" {\n riscv.directive \".globl\" \"matmul\"\n riscv.directive \".p2align\" \"2\"\n riscv_func.func @matmul(%A : !riscv.reg<a0>, %B : !riscv.reg<a1>, %C : !riscv.reg<a2>) {\n %A_1 = riscv.mv %A : (!riscv.reg<a0>) -> !riscv.reg<t5>\n %B_1 = riscv.mv %B : (!riscv.reg<a1>) -> !riscv.reg<t4>\n %C_1 = riscv.mv %C : (!riscv.reg<a2>) -> !riscv.reg<t3>\n %0 = riscv.li 2 : !riscv.reg<a3>\n %1 = riscv.li 2 : !riscv.reg<t0>\n %2 = riscv.li 2 : !riscv.reg<t2>\n %3 = riscv.get_register : !riscv.reg<zero>\n %4 = riscv.mv %3 : (!riscv.reg<zero>) -> !riscv.reg<t6>\n riscv_cf.branch ^0(%4 : !riscv.reg<t6>) attributes {comment = \"Constant folded riscv_cf.bge\"}\n ^0(%5 : !riscv.reg<t6>):\n riscv.label \"scf_body_2_for\"\n %6 = riscv.mv %3 : (!riscv.reg<zero>) -> !riscv.reg<a4>\n riscv_cf.branch ^1(%6 : !riscv.reg<a4>) attributes {comment = \"Constant folded riscv_cf.bge\"}\n ^1(%7 : !riscv.reg<a4>):\n riscv.label \"scf_body_1_for\"\n %8 = riscv.mv %3 : (!riscv.reg<zero>) -> !riscv.reg<a5>\n riscv_cf.branch ^2(%8 : !riscv.reg<a5>) attributes {comment = \"Constant folded riscv_cf.bge\"}\n ^2(%9 : !riscv.reg<a5>):\n riscv.label \"scf_body_0_for\"\n %pointer_dim_stride = riscv.li 2 : !riscv.reg<a6>\n %pointer_dim_offset = riscv.mul %5, %pointer_dim_stride : (!riscv.reg<t6>, !riscv.reg<a6>) -> !riscv.reg<a6>\n %pointer_offset = riscv.add %pointer_dim_offset, %9 : (!riscv.reg<a6>, !riscv.reg<a5>) -> !riscv.reg<a6>\n %bytes_per_element = riscv.li 8 : !riscv.reg<a7>\n %scaled_pointer_offset = riscv.mul %pointer_offset, %bytes_per_element {comment = \"multiply by element size\"} : (!riscv.reg<a6>, !riscv.reg<a7>) -> !riscv.reg<a6>\n %offset_pointer = riscv.add %A_1, %scaled_pointer_offset : (!riscv.reg<t5>, !riscv.reg<a6>) -> !riscv.reg<a6>\n %a = riscv.fld %offset_pointer, 0 {comment = \"load double from memref of shape (2, 2)\"} : (!riscv.reg<a6>) -> !riscv.freg<ft1>\n %pointer_dim_stride_1 = riscv.li 2 : !riscv.reg<a6>\n %pointer_dim_offset_1 = riscv.mul %9, %pointer_dim_stride_1 : (!riscv.reg<a5>, !riscv.reg<a6>) -> !riscv.reg<a6>\n %pointer_offset_1 = riscv.add %pointer_dim_offset_1, %7 : (!riscv.reg<a6>, !riscv.reg<a4>) -> !riscv.reg<a6>\n %bytes_per_element_1 = riscv.li 8 : !riscv.reg<a7>\n %scaled_pointer_offset_1 = riscv.mul %pointer_offset_1, %bytes_per_element_1 {comment = \"multiply by element size\"} : (!riscv.reg<a6>, !riscv.reg<a7>) -> !riscv.reg<a6>\n %offset_pointer_1 = riscv.add %B_1, %scaled_pointer_offset_1 : (!riscv.reg<t4>, !riscv.reg<a6>) -> !riscv.reg<a6>\n %b = riscv.fld %offset_pointer_1, 0 {comment = \"load double from memref of shape (2, 2)\"} : (!riscv.reg<a6>) -> !riscv.freg<ft2>\n %pointer_dim_stride_2 = riscv.li 2 : !riscv.reg<a6>\n %pointer_dim_offset_2 = riscv.mul %5, %pointer_dim_stride_2 : (!riscv.reg<t6>, !riscv.reg<a6>) -> !riscv.reg<a6>\n %pointer_offset_2 = riscv.add %pointer_dim_offset_2, %7 : (!riscv.reg<a6>, !riscv.reg<a4>) -> !riscv.reg<a6>\n %bytes_per_element_2 = riscv.li 8 : !riscv.reg<a7>\n %scaled_pointer_offset_2 = riscv.mul %pointer_offset_2, %bytes_per_element_2 {comment = \"multiply by element size\"} : (!riscv.reg<a6>, !riscv.reg<a7>) -> !riscv.reg<a6>\n %offset_pointer_2 = riscv.add %C_1, %scaled_pointer_offset_2 : (!riscv.reg<t3>, !riscv.reg<a6>) -> !riscv.reg<a6>\n %acc_old = riscv.fld %offset_pointer_2, 0 {comment = \"load double from memref of shape (2, 2)\"} : (!riscv.reg<a6>) -> !riscv.freg<ft0>\n %prod = riscv.fmul.d %a, %b : (!riscv.freg<ft1>, !riscv.freg<ft2>) -> !riscv.freg<ft1>\n %acc_new = riscv.fadd.d %acc_old, %prod : (!riscv.freg<ft0>, !riscv.freg<ft1>) -> !riscv.freg<ft0>\n %pointer_dim_stride_3 = riscv.li 2 : !riscv.reg<a6>\n %pointer_dim_offset_3 = riscv.mul %5, %pointer_dim_stride_3 : (!riscv.reg<t6>, !riscv.reg<a6>) -> !riscv.reg<a6>\n %pointer_offset_3 = riscv.add %pointer_dim_offset_3, %7 : (!riscv.reg<a6>, !riscv.reg<a4>) -> !riscv.reg<a6>\n %by

This representation of the program in xDSL corresponds ~1:1 to RISC-V assembly, and we can use a helper function to print that out.

In [ ]:
riscv_asm = riscv_code(riscv_asm_module)

mo.md(f"""\
**RISC-V Assembly:**

{xmo.asm_html(riscv_asm)}
"""
)

RISC-V Assembly: 
 <marimo-code-editor data-initial-value='".text\n.globl matmul\n.p2align 2\nmatmul:\n mv t5, a0\n mv t4, a1\n mv t3, a2\n li a3, 2\n li t0, 2\n li t2, 2\n mv t6, zero\n # Constant folded riscv_cf.bge\nscf_body_2_for:\n mv a4, zero\n # Constant folded riscv_cf.bge\nscf_body_1_for:\n mv a5, zero\n # Constant folded riscv_cf.bge\nscf_body_0_for:\n li a6, 2\n mul a6, t6, a6\n add a6, a6, a5\n li a7, 8\n mul a6, a6, a7 # multiply by element size\n add a6, t5, a6\n fld ft1, 0(a6) # load double from memref of shape (2, 2)\n li a6, 2\n mul a6, a5, a6\n add a6, a6, a4\n li a7, 8\n mul a6, a6, a7 # multiply by element size\n add a6, t4, a6\n fld ft2, 0(a6) # load double from memref of shape (2, 2)\n li a6, 2\n mul a6, t6, a6\n add a6, a6, a4\n li a7, 8\n mul a6, a6, a7 # multiply by element size\n add a6, t3, a6\n fld ft0, 0(a6) # load double from memref of shape (2, 2)\n fmul.d ft1, ft1, ft2\n fadd.d ft0, ft0, ft1\n li a6, 2\n mul a6, t6, a6\n add a6, a6, a4\n li a7, 8\n mul a6, a6, a7 # multiply by element size\n add a6, t3, a6\n fsd ft0, 0(a6) # store double value to memref of shape (2, 2)\n addi a5, a5, 1\n blt a5, t2, scf_body_0_for\nscf_body_end_0_for:\n addi a4, a4, 1\n blt a4, t0, scf_body_1_for\nscf_body_end_1_for:\n addi t6, t6, 1\n blt t6, a3, scf_body_2_for\nscf_body_end_2_for:\n ret\n"' data-label='null' data-language='"python"' data-placeholder='""' data-disabled='true'>

### Compiling to Snitch

xDSL is also capable of targeting Snitch, and making use of its streaming registers and fixed-repetition loop. We use a different lowering flow from the linalg.generic representation to represent a high-level, structured, but Snitch-specific representation of the code:

In [ ]:
from xdsl.transforms.test_lower_linalg_to_snitch import LOWER_MEMREF_STREAM_TO_SNITCH_STREAM_PASSES, OPTIMISE_MEMREF_STREAM_PASSES

convert_linalg_to_snitch = PipelinePass(
    [
        convert_linalg_to_memref_stream.ConvertLinalgToMemrefStreamPass(),
        arith_add_fastmath.AddArithFastMathFlagsPass(),
        *OPTIMISE_MEMREF_STREAM_PASSES,
        *LOWER_MEMREF_STREAM_TO_SNITCH_STREAM_PASSES,
        convert_riscv_scf_for_to_frep.ConvertRiscvScfForToFrepPass(),
    ]
)

snitch_stream_ctx, snitch_stream_module, snitch_stream_html = xmo.pipeline_html(
    linalg_ctx, linalg_module, tuple(("", p) for p in convert_linalg_to_snitch.passes),
)

snitch_stream_html

convert-linalg-to-memref-stream <marimo-code-editor data-initial-value='"builtin.module {\n func.func @matmul(%A : memref<2x2xf64>, %B : memref<2x2xf64>, %C : memref<2x2xf64>) {\n memref_stream.generic {\n bounds = [2, 2, 2],\n indexing_maps = [\n affine_map<(d0, d1, d2) -> (d0, d2)>,\n affine_map<(d0, d1, d2) -> (d2, d1)>,\n affine_map<(d0, d1, d2) -> (d0, d1)>\n ],\n iterator_types = [\"parallel\", \"parallel\", \"reduction\"]\n } ins(%A, %B : memref<2x2xf64>, memref<2x2xf64>) outs(%C : memref<2x2xf64>) {\n ^0(%a : f64, %b : f64, %acc_old : f64):\n %prod = arith.mulf %a, %b : f64\n %acc_new = arith.addf %acc_old, %prod : f64\n memref_stream.yield %acc_new : f64\n }\n func.return\n }\n}"' data-label='null' data-language='"javascript"' data-placeholder='""' data-disabled='true'> arith-add-fastmath <marimo-code-editor data-initial-value='"builtin.module {\n func.func @matmul(%A : memref<2x2xf64>, %B : memref<2x2xf64>, %C : memref<2x2xf64>) {\n memref_stream.generic {\n bounds = [2, 2, 2],\n indexing_maps = [\n affine_map<(d0, d1, d2) -> (d0, d2)>,\n affine_map<(d0, d1, d2) -> (d2, d1)>,\n affine_map<(d0, d1, d2) -> (d0, d1)>\n ],\n iterator_types = [\"parallel\", \"parallel\", \"reduction\"]\n } ins(%A, %B : memref<2x2xf64>, memref<2x2xf64>) outs(%C : memref<2x2xf64>) {\n ^0(%a : f64, %b : f64, %acc_old : f64):\n %prod = arith.mulf %a, %b fastmath<fast> : f64\n %acc_new = arith.addf %acc_old, %prod fastmath<fast> : f64\n memref_stream.yield %acc_new : f64\n }\n func.return\n }\n}"' data-label='null' data-language='"javascript"' data-placeholder='""' data-disabled='true'> canonicalize (1) <marimo-code-editor data-initial-value='"builtin.module {\n func.func @matmul(%A : memref<2x2xf64>, %B : memref<2x2xf64>, %C : memref<2x2xf64>) {\n memref_stream.generic {\n bounds = [2, 2, 2],\n indexing_maps = [\n affine_map<(d0, d1, d2) -> (d0, d2)>,\n affine_map<(d0, d1, d2) -> (d2, d1)>,\n affine_map<(d0, d1, d2) -> (d0, d1)>\n ],\n iterator_types = [\"parallel\", \"parallel\", \"reduction\"]\n } ins(%A, %B : memref<2x2xf64>, memref<2x2xf64>) outs(%C : memref<2x2xf64>) {\n ^0(%a : f64, %b : f64, %acc_old : f64):\n %prod = arith.mulf %a, %b fastmath<fast> : f64\n %acc_new = arith.addf %acc_old, %prod fastmath<fast> : f64\n memref_stream.yield %acc_new : f64\n }\n func.return\n }\n}"' data-label='null' data-language='"javascript"' data-placeholder='""' data-disabled='true'> memref-stream-infer-fill <marimo-code-editor data-initial-value='"builtin.module {\n func.func @matmul(%A : memref<2x2xf64>, %B : memref<2x2xf64>, %C : memref<2x2xf64>) {\n memref_stream.generic {\n bounds = [2, 2, 2],\n indexing_maps = [\n affine_map<(d0, d1, d2) -> (d0, d2)>,\n affine_map<(d0, d1, d2) -> (d2, d1)>,\n affine_map<(d0, d1, d2) -> (d0, d1)>\n ],\n iterator_types = [\"parallel\", \"parallel\", \"reduction\"]\n } ins(%A, %B : memref<2x2xf64>, memref<2x2xf64>) outs(%C : memref<2x2xf64>) {\n ^0(%a : f64, %b : f64, %acc_old : f64):\n %prod = arith.mulf %a, %b fastmath<fast> : f64\n %acc_new = arith.addf %acc_old, %prod fastmath<fast> : f64\n memref_stream.yield %acc_new : f64\n }\n func.return\n }\n}"' data-label='null' data-language='"javascript"' data-placeholder='""' data-disabled='true'> memref-stream-unnest-out-parameters <marimo-code-editor data-initial-value='"builtin.module {\n func.func @matmul(%A : memref<2x2xf64>, %B : memref<2x2xf64>, %C : memref<2x2xf64>) {\n memref_stream.generic {\n bounds = [2, 2, 2],\n indexing_maps = [\n affine_map<(d0, d1, d2) -> (d0, d2)>,\n affine_map<(d0, d1, d2) -> (d2, d1)>,\n affine_map<(d0, d1) -> (d0, d1)>\n ],\n iterator_types = [\"parallel\", \"parallel\", \"reduction\"]\n } ins(%A, %B : memref<2x2xf64>, memref<2x2xf64>) outs(%C : memref<2x2xf64>) {\n ^0(%a : f64, %b : f64, %acc_old : f64):\n %prod = arith.mulf %a, %b fastmath<fast> : f64\n %acc_new = arith.addf %acc_old, %prod fastmath<fast> : f64\n memref_stream.yield %acc_new : f64\n }\n func.return\n }\n}"' data-label='null' data-language='"javascript"' data-plac

We can then lower this to assembly that includes assembly instructions from the Snitch-extended ISA:

In [ ]:
from xdsl.transforms.test_lower_linalg_to_snitch import LOWER_SNITCH_STREAM_TO_ASM_PASSES

snitch_asm_ctx, snitch_asm_module, snitch_asm_html = xmo.pipeline_html(
    snitch_stream_ctx, snitch_stream_module, tuple(("", p) for p in LOWER_SNITCH_STREAM_TO_ASM_PASSES)
)

snitch_asm_html

canonicalize (1) <marimo-code-editor data-initial-value='"builtin.module {\n riscv.assembly_section \".text\" {\n riscv.directive \".globl\" \"matmul\"\n riscv.directive \".p2align\" \"2\"\n riscv_func.func @matmul(%A : !riscv.reg<a0>, %B : !riscv.reg<a1>, %C : !riscv.reg<a2>) {\n %A_1 = riscv.mv %A : (!riscv.reg<a0>) -> !riscv.reg\n %B_1 = riscv.mv %B : (!riscv.reg<a1>) -> !riscv.reg\n %C_1 = riscv.mv %C : (!riscv.reg<a2>) -> !riscv.reg\n snitch_stream.streaming_region {\n patterns = [\n #snitch_stream.stride_pattern<ub = [4], strides = [8], repeat = 2>,\n #snitch_stream.stride_pattern<ub = [2, 2, 2], strides = [0, 16, 8]>\n ]\n } ins(%A_1, %B_1 : !riscv.reg, !riscv.reg) {\n ^0(%a : !snitch.readable<!riscv.freg>, %b : !snitch.readable<!riscv.freg>):\n %0 = riscv.li 2 : !riscv.reg\n %1 = riscv.get_register : !riscv.reg<zero>\n %2 = riscv.mv %1 : (!riscv.reg<zero>) -> !riscv.reg\n %3 = riscv.li 1 : !riscv.reg\n riscv_scf.for %4 : !riscv.reg = %2 to %0 step %3 {\n %pointer_dim_stride = riscv.li 2 : !riscv.reg\n %pointer_dim_offset = riscv.mul %4, %pointer_dim_stride : (!riscv.reg, !riscv.reg) -> !riscv.reg\n %pointer_offset = riscv.mv %pointer_dim_offset : (!riscv.reg) -> !riscv.reg\n %bytes_per_element = riscv.li 8 : !riscv.reg\n %scaled_pointer_offset = riscv.mul %pointer_offset, %bytes_per_element {comment = \"multiply by element size\"} : (!riscv.reg, !riscv.reg) -> !riscv.reg\n %offset_pointer = riscv.add %C_1, %scaled_pointer_offset : (!riscv.reg, !riscv.reg) -> !riscv.reg\n %5 = riscv.fld %offset_pointer, 0 {comment = \"load double from memref of shape (2, 2)\"} : (!riscv.reg) -> !riscv.freg\n %pointer_dim_stride_1 = riscv.li 2 : !riscv.reg\n %pointer_dim_offset_1 = riscv.mul %4, %pointer_dim_stride_1 : (!riscv.reg, !riscv.reg) -> !riscv.reg\n %pointer_offset_1 = riscv.addi %pointer_dim_offset_1, 1 : (!riscv.reg) -> !riscv.reg\n %bytes_per_element_1 = riscv.li 8 : !riscv.reg\n %scaled_pointer_offset_1 = riscv.mul %pointer_offset_1, %bytes_per_element_1 {comment = \"multiply by element size\"} : (!riscv.reg, !riscv.reg) -> !riscv.reg\n %offset_pointer_1 = riscv.add %C_1, %scaled_pointer_offset_1 : (!riscv.reg, !riscv.reg) -> !riscv.reg\n %6 = riscv.fld %offset_pointer_1, 0 {comment = \"load double from memref of shape (2, 2)\"} : (!riscv.reg) -> !riscv.freg\n %7 = riscv.fmv.d %5 : (!riscv.freg) -> !riscv.freg\n %8 = riscv.fmv.d %6 : (!riscv.freg) -> !riscv.freg\n %9 = riscv.li 1 : !riscv.reg\n %10, %11 = riscv_snitch.frep_outer %9 iter_args(%acc_old = %7, %acc_old_1 = %8) -> (!riscv.freg, !riscv.freg) {\n %a_1 = riscv_snitch.read from %a : !riscv.freg\n %a_2 = riscv_snitch.read from %a : !riscv.freg\n %b_1 = riscv_snitch.read from %b : !riscv.freg\n %b_2 = riscv_snitch.read from %b : !riscv.freg\n %acc_new = riscv.fmadd.d %a_1, %b_1, %acc_old : (!riscv.freg, !riscv.freg, !riscv.freg) -> !riscv.freg\n %acc_new_1 = riscv.fmadd.d %a_2, %b_2, %acc_old_1 : (!riscv.freg, !riscv.freg, !riscv.freg) -> !riscv.freg\n riscv_snitch.frep_yield %acc_new, %acc_new_1 : !riscv.freg, !riscv.freg\n }\n %12 = riscv.fmv.d %10 : (!riscv.freg) -> !riscv.freg\n %13 = riscv.fmv.d %11 : (!riscv.freg) -> !riscv.freg\n %pointer_dim_stride_2 = riscv.li 2 : !riscv.reg\n %pointer_dim_offset_2 = riscv.mul %4, %pointer_dim_stride_2 : (!riscv.reg, !riscv.reg) -> !riscv.reg\n %pointer_offset_2 = riscv.mv %pointer_dim_offset_2 : (!riscv.reg) -> !riscv.reg\n %bytes_per_element_2 = riscv.li 8 : !riscv.reg\n %scaled_pointer_offset_2 = riscv.mul %pointer_offset_2, %bytes_per_element_2 {comment = \"multiply by element size\"} : (!riscv.reg, !riscv.reg) -> !riscv.reg\n %offset_pointer_2 = riscv.add %C_1, %scaled_pointer_offset_2 : (!riscv.reg, !riscv.reg) -> !riscv.reg\n riscv.fsd %offset_pointer_2, %12, 0 {comment = \"store double value to memref of shape (2, 2)\"} : (!riscv.reg, !riscv.freg) -> ()\n %pointer_dim_stride_3 = riscv.li 2 : !riscv.reg\n %pointer_dim_offset_3 = riscv.mul %4, %pointer_dim_stride_3 : (!riscv.reg, !riscv.reg) -> !riscv.reg\n %pointer_off

In [ ]:
mo.md(
    f"""
    We can see how changing our input sizes affects the assembly produced:

    {m}{m.value}

    {n}{n.value}

    {k}{k.value}
    """
)

We can see how changing our input sizes affects the assembly produced: 
 2 
 2 
 2

In [ ]:
snitch_asm = riscv_code(snitch_asm_module)

mo.md(f"""\
**Snitch Assembly:**

{xmo.asm_html(snitch_asm)}
"""
)

Snitch Assembly: 
 <marimo-code-editor data-initial-value='".text\n.globl matmul\n.p2align 2\n # Regalloc stats: {\"preallocated_float\": [\"ft0\", \"ft1\", \"ft2\"], \"preallocated_int\": [\"a0\", \"a1\", \"a2\", \"zero\"], \"allocated_float\": [\"ft0\", \"ft1\", \"ft3\", \"ft4\"], \"allocated_int\": [\"a0\", \"a1\", \"a2\", \"t0\", \"t1\", \"t2\", \"t3\", \"t4\", \"t5\", \"zero\"]}\nmatmul:\n mv t3, a0\n mv t2, a1\n mv t0, a2\n li t1, 3\n scfgwi t1, 64 # dm 0 dim 0 bound\n li t1, 8\n scfgwi t1, 192 # dm 0 dim 0 stride\n li t1, 1\n scfgwi t1, 32 # dm 0 repeat\n li t1, 1\n scfgwi t1, 65 # dm 1 dim 0 bound\n li t1, 1\n scfgwi t1, 97 # dm 1 dim 1 bound\n li t1, 1\n scfgwi t1, 129 # dm 1 dim 2 bound\n li t1, 8\n scfgwi t1, 193 # dm 1 dim 0 stride\n li t1, 8\n scfgwi t1, 225 # dm 1 dim 1 stride\n li t1, -24\n scfgwi t1, 257 # dm 1 dim 2 stride\n scfgwi zero, 33 # dm 1 repeat\n scfgwi t3, 768 # dm 0 dim 0 source\n scfgwi t2, 833 # dm 1 dim 2 source\n csrrsi zero, 1984, 1 # SSR enable\n li t2, 2\n mv t1, zero\n # Constant folded riscv_cf.bge\nscf_body_0_for:\n li t4, 2\n mul t4, t1, t4\n li t5, 8\n mul t4, t4, t5 # multiply by element size\n add t4, t0, t4\n fld ft4, 0(t4) # load double from memref of shape (2, 2)\n li t4, 2\n mul t4, t1, t4\n addi t4, t4, 1\n li t5, 8\n mul t4, t4, t5 # multiply by element size\n add t4, t0, t4\n fld ft3, 0(t4) # load double from memref of shape (2, 2)\n li t4, 1\n frep.o t4, 2, 0, 0\n fmadd.d ft4, ft0, ft1, ft4\n fmadd.d ft3, ft0, ft1, ft3\n li t4, 2\n mul t4, t1, t4\n li t5, 8\n mul t4, t4, t5 # multiply by element size\n add t4, t0, t4\n fsd ft4, 0(t4) # store double value to memref of shape (2, 2)\n li t4, 2\n mul t4, t1, t4\n addi t4, t4, 1\n li t5, 8\n mul t4, t4, t5 # multiply by element size\n add t4, t0, t4\n fsd ft3, 0(t4) # store double value to memref of shape (2, 2)\n addi t1, t1, 1\n blt t1, t2, scf_body_0_for\nscf_body_end_0_for:\n csrrci zero, 1984, 1 # SSR disable\n ret\n"' data-label='null' data-language='"python"' data-placeholder='""' data-disabled='true'>

### Interpreting the assembly using xDSL

One of the useful features of xDSL is its interpreter. Here we've implemented all the necessary functions to interpret the code at a low level, to check that our compilation is correct. Here's the slider modifying the shape variable defined above, we can slide it to see the result of the code compiled with different input shapes, and interpreted at the RISC-V level.

In [ ]:
from math import prod

from xdsl.interpreter import Interpreter, OpCounter
from xdsl.interpreters import register_implementations
from xdsl.interpreters.shaped_array import ShapedArray

a_len = prod(a_shape)
b_len = prod(b_shape)
c_len = prod(c_shape)

a_shaped = ShapedArray(TypedPtr.new_float64([i + 1 for i in range(a_len)]), a_shape)
b_shaped = ShapedArray(TypedPtr.new_float64([(i + 1) / 100 for i in range(b_len)]), b_shape)
riscv_c_shaped = ShapedArray(TypedPtr.new_float64([0.0] * c_len), c_shape)

riscv_op_counter = OpCounter()
riscv_interpreter = Interpreter(riscv_module, listeners=(riscv_op_counter,))

register_implementations(riscv_interpreter, riscv_ctx, include_onnx=False)

riscv_interpreter.call_op("matmul", (a_shaped.data_ptr.raw, b_shaped.data_ptr.raw, riscv_c_shaped.data_ptr.raw))

mo.md(f"""
**RISC-V Results:**

A: {a_shaped}

B: {b_shaped}

C: {riscv_c_shaped}
""")

RISC-V Results: 
 A: [[1.0, 2.0], [3.0, 4.0]] 
 B: [[0.01, 0.02], [0.03, 0.04]] 
 C: [[0.06999999999999999, 0.1], [0.15, 0.22]]

In [ ]:
snitch_op_counter = OpCounter()
snitch_interpreter = Interpreter(
    snitch_stream_module, listeners=(snitch_op_counter,)
)

snitch_c_shaped = ShapedArray(TypedPtr.new_float64([0.0] * c_len), c_shape)

register_implementations(snitch_interpreter, snitch_stream_ctx, include_onnx=False)

snitch_interpreter.call_op(
    "matmul",
    (
        a_shaped.data_ptr.raw,
        b_shaped.data_ptr.raw,
        snitch_c_shaped.data_ptr.raw,
    ),
)

mo.md(f"""

**Snitch Results:**

A: {a_shaped}

B: {b_shaped}

C: {riscv_c_shaped}
""")

Snitch Results: 
 A: [[1.0, 2.0], [3.0, 4.0]] 
 B: [[0.01, 0.02], [0.03, 0.04]] 
 C: [[0.06999999999999999, 0.1], [0.15, 0.22]]

In [ ]:
rv_dict = dict(riscv_op_counter.ops)
sn_dict = dict(snitch_op_counter.ops)

all_keys = sorted(set(rv_dict) | set(sn_dict))
max_len_key = max(len(k) for k in all_keys)
max_len_value = max(len(str(val)) for val in (*rv_dict.values(), *sn_dict.values()))

def format_row(key: str, *values: str):
    paddings = tuple(" " * (max_len_value - len(val)) for val in values)
    vals = "".join(f"\t{padding}{val}" for padding, val in zip(paddings, values))
    return f"{key}{' ' * (max_len_key - len(key))}{vals}\n"

rows = " " * max_len_key + "\trv\tsn\tdiff\n"

ZERO_VAL = "."

for key in all_keys:
    rv_val = rv_dict.get(key, 0)
    sn_val = sn_dict.get(key, 0)
    diff_val = sn_val - rv_val

    rv_str = str(rv_val) if rv_val else ZERO_VAL
    sn_str = str(sn_val) if sn_val else ZERO_VAL
    diff_str = (
        (f"+{diff_val}" if diff_val > 0 else f"{diff_val}") if diff_val else "="
    )

    rows += key
    rows += " " * (max_len_key - len(key))
    rows += "\t"
    rows += " " * (max_len_value - len(rv_str))
    rows += rv_str
    rows += "\t"
    rows += " " * (max_len_value - len(sn_str))
    rows += sn_str
    rows += "\t"
    rows += " " * (max_len_value - len(diff_str))
    rows += diff_str
    rows += "\n"

rv_sum = sum(rv_dict.values())
sn_sum = sum(sn_dict.values())
total_diff = sn_sum - rv_sum

rows += format_row("total", str(rv_sum), str(sn_sum), str(total_diff))

mo.md(
    f"""
The interpreter kept track of the number of times an operation was executed, which we can use as a proxy for performance.

For example, we can see that one version has many more instructions executed overall ({rv_sum} vs {sn_sum}), and that one version uses explicit load and store instructions, while the other uses the streaming equivalents:

{m}{m.value}

{n}{n.value}

{k}{k.value}

```
{rows}
```
"""
)

The interpreter kept track of the number of times an operation was executed, which we can use as a proxy for performance. 
 For example, we can see that one version has many more instructions executed overall (270 vs 144), and that one version uses explicit load and store instructions, while the other uses the streaming equivalents: 
 2 
 2 
 2 
 rv sn diff
riscv.add 64 24 -40
riscv.addi . 2 +2
riscv.fadd.d 8 8 =
riscv.fld 24 4 -20
riscv.fmul.d 8 8 =
riscv.fmv.d . 8 +8
riscv.fsd 8 4 -4
riscv.li 69 30 -39
riscv.mul 64 24 -40
riscv.mv 3 3 =
riscv.sub . 2 +2
riscv_func.return 1 1 =
riscv_scf.for 7 1 -6
riscv_scf.yield 14 2 -12
riscv_snitch.frep_outer . 2 +2
riscv_snitch.frep_yield . 4 +4
riscv_snitch.read . 16 +16
snitch_stream.streaming_region . 1 +1
total 270 144 -126